https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis

este es otro https://www.kaggle.com/code/aslanahmedov/market-basket-analysis-apriori-algorithm/notebook

About Dataset
Market Basket Analysis
Market basket analysis with Apriori algorithm

The retailer wants to target customers with suggestions on itemset that a customer is most likely to purchase .I was given dataset contains data of a retailer; the transaction data provides data around all the transactions that have happened over a period of time. Retailer will use result to grove in his industry and provide for customer suggestions on itemset, we be able increase customer engagement and improve customer experience and identify customer behavior. I will solve this problem with use Association Rules type of unsupervised learning technique that checks for the dependency of one data item on another data item.

Introduction
Association Rule is most used when you are planning to build association in different objects in a set. It works when you are planning to find frequent patterns in a transaction database. It can tell you what items do customers frequently buy together and it allows retailer to identify relationships between the items.

An Example of Association Rules
Assume there are 100 customers, 10 of them bought Computer Mouth, 9 bought Mat for Mouse and 8 bought both of them.

bought Computer Mouth => bought Mat for Mouse
support = P(Mouth & Mat) = 8/100 = 0.08
confidence = support/P(Mat for Mouse) = 0.08/0.09 = 0.89
lift = confidence/P(Computer Mouth) = 0.89/0.10 = 8.9
This just simple example. In practice, a rule needs the support of several hundred transactions, before it can be considered statistically significant, and datasets often contain thousands or millions of transactions.
Strategy
Data Import
Data Understanding and Exploration
Transformation of the data – so that is ready to be consumed by the association rules algorithm
Running association rules
Exploring the rules generated
Filtering the generated rules
Visualization of Rule
Dataset Description
File name: Assignment-1_Data

List name: retaildata

File format: . xlsx

Number of Row: 522065

Number of Attributes: 7

BillNo: 6-digit number assigned to each transaction. Nominal.
Itemname: Product name. Nominal.
Quantity: The quantities of each product per transaction. Numeric.
Date: The day and time when each transaction was generated. Numeric.
Price: Product price. Numeric.
CustomerID: 5-digit number assigned to each customer. Nominal.
Country: Name of the country where each customer resides. Nominal.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

from plotly.offline import plot, iplot, init_notebook_mode
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances

from funpymodeling.exploratory import freq_tbl
from plotly.offline import plot, iplot, init_notebook_mode

In [3]:
dataset=pd.read_csv('../data/raw/Assignment-1_Data.csv',sep=';', date_parser=True)

/tmp/ipykernel_65207/2746298970.py:1: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



### Exploratory Data Analysis

In [4]:
dataset.sample(5)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
309529,565151,LUNCH BAG VINTAGE DOILY,4,01.09.2011 14:29,"1,65",16923.0,United Kingdom
430384,574950,SET OF 16 VINTAGE ROSE CUTLERY,1,08.11.2011 09:29,"24,96",NaN,United Kingdom
379594,570919,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",4,13.10.2011 10:57,"4,95",12584.0,Italy
229418,557888,ASSORTED TUTTI FRUTTI PEN,2,23.06.2011 13:51,"0,85",14656.0,United Kingdom
340876,567866,RETROSPOT CHILDRENS APRON,3,22.09.2011 14:13,"1,95",16669.0,United Kingdom


In [22]:
dataset.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [23]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [6]:
dataset.describe()

,Quantity,CustomerID
count,522064.000000,388023.000000
mean,10.090435,15316.931710
std,161.110525,1721.846964
min,-9600.000000,12346.000000
25%,1.000000,13950.000000
50%,3.000000,15265.000000
75%,10.000000,16837.000000
max,80995.000000,18287.000000


In [27]:
#Checking for missing values
#No missing values in our data
dataset.isna().sum()

BillNo             0
Itemname        1455
Quantity           0
Date               0
Price              0
CustomerID    134041
Country            0
dtype: int64

In [7]:
#dataset['Price'] = dataset['Price'].astype('int64')
#dataset['CustomerID'] = dataset['CustomerID'].astype('int64')

In [28]:
#Extracting date information into Day,Month,Year
def split_date(data):
    data['Date']=pd.to_datetime(data['Date'])
    data['Year']=data.Date.dt.year
    data['Month']=data.Date.dt.month
    data['Day']=data.Date.dt.day

In [29]:
#Spliting our date data into day ,month and year
split_date(dataset)
#check how now our data looks like
dataset.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country,Year,Month,Day
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,"2,55",17850.0,United Kingdom,2010,1,12
1,536365,WHITE METAL LANTERN,6,2010-01-12 08:26:00,"3,39",17850.0,United Kingdom,2010,1,12
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,"2,75",17850.0,United Kingdom,2010,1,12
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,"3,39",17850.0,United Kingdom,2010,1,12
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,"3,39",17850.0,United Kingdom,2010,1,12


In [31]:
#all our data is for the  Year 2010 and 2011
dataset.Year.value_counts()

2011    480822
2010     41242
Name: Year, dtype: int64

In [33]:
#Checking the data across the months (data available for the months of JAN-DEC)
dataset.Month.value_counts()

11    71148
10    54819
9     49324
7     44898
6     44085
8     43312
5     41138
1     38260
3     37326
4     36218
2     32814
12    28722
Name: Month, dtype: int64


### Analisis de la cesta

In [8]:
dataset=pd.read_csv('../data/raw/Assignment-1_Data.csv',sep=';',header = None)

/tmp/ipykernel_65207/642597484.py:1: DtypeWarning:

Columns (0,2,5) have mixed types. Specify dtype option on import or set low_memory=False.



In [9]:
dataset.describe()

,0,1,2,3,4,5,6
count,522065,520610,522065,522065,522065,388024.0,522065
unique,21667,4186,955,19642,1286,5885.0,31
top,573585,WHITE HANGING HEART T-LIGHT HOLDER,1,31.10.2011 14:41,"1,25",17841.0,United Kingdom
freq,1114,2269,106819,1114,48778,6819.0,487622


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522065 entries, 0 to 522064
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       522065 non-null  object
 1   1       520610 non-null  object
 2   2       522065 non-null  object
 3   3       522065 non-null  object
 4   4       522065 non-null  object
 5   5       388024 non-null  object
 6   6       522065 non-null  object
dtypes: object(7)
memory usage: 27.9+ MB


In [11]:
transactions = []
for i in range(0, dataset.shape[0]):
    transactions.append([str(dataset.values[i,j]) for j in range(0, 7)])

In [12]:
transactions

[['BillNo', 'Itemname', 'Quantity', 'Date', 'Price', 'CustomerID', 'Country'],
 ['536365',
  'WHITE HANGING HEART T-LIGHT HOLDER',
  '6',
  '01.12.2010 08:26',
  '2,55',
  '17850',
  'United Kingdom'],
 ['536365',
  'WHITE METAL LANTERN',
  '6',
  '01.12.2010 08:26',
  '3,39',
  '17850',
  'United Kingdom'],
 ['536365',
  'CREAM CUPID HEARTS COAT HANGER',
  '8',
  '01.12.2010 08:26',
  '2,75',
  '17850',
  'United Kingdom'],
 ['536365',
  'KNITTED UNION FLAG HOT WATER BOTTLE',
  '6',
  '01.12.2010 08:26',
  '3,39',
  '17850',
  'United Kingdom'],
 ['536365',
  'RED WOOLLY HOTTIE WHITE HEART.',
  '6',
  '01.12.2010 08:26',
  '3,39',
  '17850',
  'United Kingdom'],
 ['536365',
  'SET 7 BABUSHKA NESTING BOXES',
  '2',
  '01.12.2010 08:26',
  '7,65',
  '17850',
  'United Kingdom'],
 ['536365',
  'GLASS STAR FROSTED T-LIGHT HOLDER',
  '6',
  '01.12.2010 08:26',
  '4,25',
  '17850',
  'United Kingdom'],
 ['536366',
  'HAND WARMER UNION JACK',
  '6',
  '01.12.2010 08:28',
  '1,85',
  '17850',

In [ ]:
#!pip install apyori

In [13]:
# Training Apriori on the dataset
from apyori import apriori
rules = apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2)

In [14]:
rules

<generator object apriori at 0x7f84858b7d10>

In [15]:
# Resultados
results = list(rules)
def inspect(results):
    rh = [tuple(result[2][0][0]) for result in results]
    lh = [tuple(result[2][0][1]) for result in results]
    supports = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts = [result[2][0][3] for result in results]
    return list(zip(rh, lh, supports, confidences, lifts))

In [16]:
#Este comamdo crea un frame para ver los datos resultados
resultDataFrame=pd.DataFrame(inspect(results),
columns=['rhs','lhs','support','confidence','lift'])

In [17]:
resultDataFrame

,rhs,lhs,support,confidence,lift
0,"(0,)","(nan,)",0.004737,0.984867,3.835874
1,"(25,)","(0,42,)",0.008846,0.913732,20.002824
2,"(0,55,)","(24,)",0.005415,0.385045,8.744116
3,"(1,45,)","(12,)",0.008003,0.336963,3.011597
4,"(1,63,)","(nan,)",0.023037,0.945891,3.684070
...,...,...,...,...,...
85,"(8,29,)","(1, United Kingdom, nan)",0.005921,0.645572,4.505818
86,"(1,63, 2)","(United Kingdom, nan)",0.004729,0.945977,3.705527
87,"(2, 2,46)","(United Kingdom, nan)",0.005781,0.942830,3.693201
88,"(2, 3,29)","(United Kingdom, nan)",0.004178,0.927296,3.632350


In [18]:
resultDataFrame.sort_values('confidence',ascending=False)

,rhs,lhs,support,confidence,lift
15,"(2,51,)","(nan,)",0.005189,1.000000,3.894816
70,"(4,21,)","(United Kingdom, nan)",0.004176,1.000000,3.917142
67,"(3,36,)","(United Kingdom, nan)",0.004045,1.000000,3.917142
60,"(2,51,)","(United Kingdom, nan)",0.005189,1.000000,3.917142
9,"(14646.0,)","(Netherlands,)",0.003065,1.000000,220.933136
...,...,...,...,...,...
65,"(5,95,)","(United Kingdom, 3)",0.003469,0.242404,3.588155
25,"(4,25,)","(4,)",0.003055,0.228477,3.182837
19,"(4,95,)","(3,)",0.007708,0.224403,3.195230
50,"(10,)","(United Kingdom, 2,08)",0.009108,0.218320,7.428115


In [19]:
resultDataFrame.sort_values('support',ascending=False)

,rhs,lhs,support,confidence,lift
14,"(2,46,)","(nan,)",0.030803,0.940905,3.664650
59,"(2,46,)","(United Kingdom, nan)",0.030803,0.940905,3.685657
26,"(4,13,)","(nan,)",0.028355,0.965245,3.759452
69,"(4,13,)","(United Kingdom, nan)",0.028355,0.965245,3.781003
47,"(1,63,)","(United Kingdom, nan)",0.023037,0.945891,3.705189
...,...,...,...,...,...
52,"(16,63,)","(United Kingdom, nan)",0.003076,0.941935,3.689696
10,"(16,63,)","(nan,)",0.003076,0.941935,3.668665
9,"(14646.0,)","(Netherlands,)",0.003065,1.000000,220.933136
24,"(4,15,)","(4,)",0.003055,0.271674,3.784600
